In [1]:
# import modules
import pandas as pd
from bs4 import BeautifulSoup
import requests
import pymongo
from splinter import Browser
from splinter.exceptions import ElementDoesNotExist

In [2]:
executable_path = {'executable_path': '/usr/local/bin/chromedriver'}
browser = Browser('chrome', **executable_path, headless=False)

Nasa information

In [20]:
url = "https://mars.nasa.gov/news/?page=0&per_page=40&order=publish_date+desc%2Ccreated_at+desc&search=&category=19%2C165%2C184%2C204&blank_scope=Latest"
browser.visit(url)
html = browser.html
soup = BeautifulSoup(html, 'html.parser')

In [54]:
# Find the latest news title and paragraph text on Nasa Site

results = soup.find_all('div', class_='image_and_description_container')

for result in results:
    title_block = soup.find_all('div', class_= 'content_title')
    paragraph_block = soup.find('div', class_='article_teaser_body')

nasa_title = title_block[1].text
nasa_paragraph = paragraph_block.text
nasa_title

"Hear Audio From NASA's Perseverance As It Travels Through Deep Space"

In [52]:
title_block[1].text

"Hear Audio From NASA's Perseverance As It Travels Through Deep Space"

In [50]:
for result in title_block:
    titles = soup.find_all('div', class_= 'content_title')
titles[1]

<div class="content_title"><a href="/news/8801/hear-audio-from-nasas-perseverance-as-it-travels-through-deep-space/" target="_self">Hear Audio From NASA's Perseverance As It Travels Through Deep Space</a></div>

In [44]:

for result in title_block:
    title = result.find('a')
title

<a href="/news/8695/the-launch-is-approaching-for-nasas-next-mars-rover-perseverance/">
The Launch Is Approaching for NASA's Next Mars Rover, Perseverance
</a>

Featured image

In [57]:
# Find current featured image URL and store in variable
feature_url = "https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars"
browser.visit(feature_url)
html = browser.html
soup = BeautifulSoup(html, 'html.parser')

In [89]:
# Obtaining the section of html the URL lives in
image_results = soup.find('div',class_="carousel_container")

# Cleaning the URL to be stored in variable
url_string = image_results.article["style"].\
strip("background-image: url").\
strip("(").strip(");").strip("''")
url_string

# Creating the full URL
feature_image_url = f"https://www.jpl.nasa.gov{url_string}"

Facts Table

In [57]:
df

,Statistic,Measure
0,Equatorial Diameter:,"6,792 km"
1,Polar Diameter:,"6,752 km"
2,Mass:,6.39 × 10^23 kg (0.11 Earths)
3,Moons:,2 (Phobos & Deimos)
4,Orbit Distance:,"227,943,824 km (1.38 AU)"
5,Orbit Period:,687 days (1.9 years)
6,Surface Temperature:,-87 to -5 °C
7,First Record:,2nd millennium BC
8,Recorded By:,Egyptian astronomers


In [85]:
facts_url = "https://space-facts.com/mars/"

# Use Pandas to scrape the table containing facts about Mars
tables = pd.read_html(facts_url)
# Convert the data to an HTML Table string
df = tables[0]
df.columns = ["Statistic","Measure"]
df = df.set_index("Statistic")
html_table = df.to_dict()

In [83]:
    mars_table = df[0]
    mars_table = mars_table.rename({0: "", 1: "Data"}, axis =1).set_index("")

    #export as a html file
    mars_table.to_html("templates/mars_facts.html")


In [87]:
html_table

{'Measure': {'Equatorial Diameter:': '6,792 km',
  'Polar Diameter:': '6,752 km',
  'Mass:': '6.39 × 10^23 kg (0.11 Earths)',
  'Moons:': '2 (Phobos & Deimos)',
  'Orbit Distance:': '227,943,824 km (1.38 AU)',
  'Orbit Period:': '687 days (1.9 years)',
  'Surface Temperature:': '-87 to -5 °C',
  'First Record:': '2nd millennium BC',
  'Recorded By:': 'Egyptian astronomers'}}

Mars Hemispheres

In [120]:
hemisphere_url = "https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars"
browser.visit(hemisphere_url)
html = browser.html
soup = BeautifulSoup(html, 'html.parser')

# Obtain high resolution images of each of Mars hemispheres
results = soup.find_all('div',class_="item")
url_list = []
title = []
for result in results:
    url_list.append(result.a['href'])
    title.append(result.h3.text)
    
click_list = ["https://astrogeology.usgs.gov" + url for url in url_list]

In [131]:
img_urls = []
for site in click_list:
    browser.visit(site)
    html = browser.html
    soup = BeautifulSoup(html, 'html.parser')
    img_url = soup.find("div", class_ = "downloads").li.a["href"]
    img_urls.append(img_url)
img_urls

['https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/cerberus_enhanced.tif/full.jpg',
 'https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/schiaparelli_enhanced.tif/full.jpg',
 'https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/syrtis_major_enhanced.tif/full.jpg',
 'https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/valles_marineris_enhanced.tif/full.jpg']

In [132]:
# Save both the title url in a dictionary of lists.
hemisphere_image_urls = []
for url, name in zip(img_urls, title):
    hemisphere_image_urls.append({"title":name,"img_url":url})